In [6]:
from scipy.stats import wasserstein_distance as wsd
from scipy.stats import beta as beta_distribution

from ddsm import noise_factory
import torch

In [ ]:
num_cat = 9
num_samples = 500000

In [ ]:
def noise_factory(N, n_time_steps, a, b, total_time=4, order=100,
                  time_steps=1000, speed_balanced=True, logspace=False,
                  mode="independent", 
                  #mode="path", 
                  device="cuda",
                  noise_only=False,
                  boundary_mode='clamp')

In [2]:
total_time = 1.0
order = 100
steps_per_tick = 200
speed_balance = True #if True s=2/(a+b) is used; otherwise s=1
logspace = False
mode ='independent'

In [ ]:
device="cpu"

In [3]:
alpha = torch.ones(1)
beta =  (num_cat-1)*torch.ones(1)

In [4]:
print(alpha.shape)
print(beta.shape)

torch.Size([1])
torch.Size([1])


In [5]:
iteration = 5
for num_time_steps in [100,400,800,1200,1600,2000]:
    for boundary_mode in ['clamp', 'reflect_boundaries', 'reflect']:
        distances = torch.zeros(iteration)
        for i in range(iteration):
            v_one, v_zero = noise_factory(num_samples, 
                                            num_time_steps,
                                            alpha,
                                            beta,
                                            total_time=total_time,
                                            order=order,
                                            time_steps=steps_per_tick,
                                            logspace=logspace,
                                            speed_balanced=speed_balance,
                                            mode=mode,
                                            device=device,
                                            boundary_mode=boundary_mode,
                                            noise_only=True)

            vT_approx = torch.cat([v_one[:,-1,0],v_zero[:,-1,0]])
            vT_true = beta_distribution.rvs(alpha, beta, size=vT_approx.shape[0])
            d = wsd(vT_approx,vT_true)
            print(f'{boundary_mode} with steps={num_time_steps}: wsd={d}')
            distances[i] = d

        #print('v shape one', v_one.shape)
        #print('v shape zero', v_zero.shape)

        print(f'\n{boundary_mode} with steps={num_time_steps}: mean={torch.mean(distances)} and speed_balance={speed_balance}')
        print(f'{boundary_mode} with steps={num_time_steps}: std={torch.std(distances)} and speed_balance={speed_balance}\n')

clamp with steps=100: wsd=0.007017955881073237
clamp with steps=100: wsd=0.007150544142611445
clamp with steps=100: wsd=0.0068189085119101725
clamp with steps=100: wsd=0.007123989489040336
clamp with steps=100: wsd=0.006950329654692467

clamp with steps=100: mean=0.00701234582811594 and speed_balance=True
clamp with steps=100: std=0.0001349616068182513 and speed_balance=True

reflect_boundaries with steps=100: wsd=0.006987178854938313
reflect_boundaries with steps=100: wsd=0.006732128456849396
reflect_boundaries with steps=100: wsd=0.006797766843601802
reflect_boundaries with steps=100: wsd=0.0070380988440061605
reflect_boundaries with steps=100: wsd=0.006947154300943437

reflect_boundaries with steps=100: mean=0.006900465581566095 and speed_balance=True
reflect_boundaries with steps=100: std=0.00012992908887099475 and speed_balance=True

reflect with steps=100: wsd=0.007094939547602649
reflect with steps=100: wsd=0.0068946464727944815
reflect with steps=100: wsd=0.006971135644897272
r


KeyboardInterrupt



## Ground truth

In [74]:
iteration = 10
distances = torch.zeros(iteration)
for i in range(iteration):
    test1 = beta_distribution.rvs(alpha, beta, size=1000000)
    test2 = beta_distribution.rvs(alpha, beta, size=1000000)
    d = wsd(test1,test2)
    print('distance',d)
    distances[i] = d

test1 (1000000,)
test2 (1000000,)
distance 0.00015255260316868874
test1 (1000000,)
test2 (1000000,)
distance 0.00021186835176490015
test1 (1000000,)
test2 (1000000,)
distance 0.00011741920541712615
test1 (1000000,)
test2 (1000000,)
distance 0.00019672195663265597
test1 (1000000,)
test2 (1000000,)
distance 0.00014337657853888966
test1 (1000000,)
test2 (1000000,)
distance 0.00022759701250514168
test1 (1000000,)
test2 (1000000,)
distance 0.00029576271299861255
test1 (1000000,)
test2 (1000000,)
distance 0.00010474523625257931
test1 (1000000,)
test2 (1000000,)
distance 0.0001795923039257112
test1 (1000000,)
test2 (1000000,)
distance 0.00011724995625918759


In [75]:
print('mean',torch.mean(distances))
print('std',torch.std(distances))

mean tensor(0.0002)
std tensor(5.9900e-05)
